In [1]:
import numpy as np
import os
ppgs_file = os.listdir('../db1/ppgs')
mels_file = os.listdir('../db1/mels')
total_num = len(ppgs_file)

In [2]:
ppgs = np.zeros([total_num,665,256],dtype=np.float32)
mels = np.zeros([total_num,665,80],dtype=np.float32)

In [51]:
# # arr3 = np.append([[1, 2]], [[1, 2], [1, 2]],axis=0)
# # print(arr3)
# ppgs = np.array([[1,2,3],[4,5,6]])
# ppgs = ppgs.reshape(1,ppgs.shape[0],ppgs.shape[1])
# k = np.array([[7,8,9],[8,9,10]])
# k = k.reshape(1,k.shape[0],k.shape[1])
# print(ppgs.shape)
# print(k.shape)
# ppgs = np.append(ppgs,k,axis=0)
# print(ppgs.shape)

# # ppgs = np.array([range(256),range(256)])
# # print(ppgs.shape)


(1, 2, 3)
(1, 2, 3)
(2, 2, 3)


In [3]:
for i in range(total_num):
    p = np.load('../db1/ppgs/'+ppgs_file[i])
    m = (np.load('../db1/mels/'+mels_file[i]))[:p.shape[0]]
    ppgs[i] = np.pad(p,((0,665-p.shape[0]),(0,0)),'constant')
    mels[i] = np.pad(m,((0,665-p.shape[0]),(0,0)),'constant')

In [4]:
print(type(ppgs),ppgs.shape,ppgs[0].shape)

<class 'numpy.ndarray'> (10000, 665, 256) (665, 256)


In [5]:
print(type(mels),mels.shape,mels[0].shape)

<class 'numpy.ndarray'> (10000, 665, 80) (665, 80)


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(ppgs, mels, test_size=0.1)

In [7]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(9000, 665, 256) (9000, 665, 80)
(1000, 665, 256) (1000, 665, 80)


In [8]:
print(len(X_train))

9000


In [5]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import pandas as pd

In [6]:
def get_batch(input_data,output_data,batch_size):
    batch_num = len(input_data)//batch_size
    for k in range(batch_num):
        begin = k*batch_size
        end = begin + batch_size
        input_batch = input_data[begin:end]
        target_batch = output_data[begin:end]
        yield input_batch,target_batch

In [7]:
batch = get_batch(X_train,Y_train,64)
input_batch, target_batch = next(batch)
print(input_batch.shape)
print(target_batch.shape)

(64, 665, 256)
(64, 665, 80)


In [8]:
def Conv1d(inputs,filters,size,padding='same',use_bias=False,activation = None,scope='conv1d'):
    # inputs: A 3D tensor with shape of[batch, time, depth].
    with tf.variable_scope(scope):
#         conv1d_output = tf.layers.conv1d(
#       inputs,
#       filters=channels,
#       kernel_size=kernel_size,
#       activation=activation,
#       padding='same')
#         params = {"inputs":inputs,"filters":filters,"kernel_size":size,"padding":padding,"activation":activation,"use_bias": use_bias}

        conv1d_output = tf.layers.conv1d(inputs = inputs,filters=filters,kernel_size=size,activation=activation,padding='same')

    return conv1d_output

In [9]:
def conv1d_banks(inputs, num_units = None,K=16,is_training=True,scope="conv1d_banks"):
    with tf.variable_scope(scope):
#         outputs = conv1d(inputs = inputs,filters = num_units,size = 1,activation = tf.nn.relu)
#         for k in range(2,K+1):
#             with tf.variable_scope("num_{}".format(k)):
#                 output = conv1d(inputs = inputs,filters = num_units,size = k,activation = tf.nn.relu, is_training = is_training)
#                 outputs = tf.concat((outputs,output),-1)
                
        conv_outputs = tf.concat([Conv1d(inputs=inputs,filters = 128,size = k, activation = tf.nn.relu,scope = 'conv1d_%d' % k) for k in range(1, K+1)],axis=-1)
#         outputs = normalize(outputs,is_training=is_training,activation_fn=tf.nn.relu)
        outputs = tf.layers.batch_normalization(conv_outputs,training=is_training)

    return outputs

In [10]:
def gru_bi(inputs,num_units=128,seqlen=None,scope='gru'):
    with tf.variable_scope(scope):
        cell = tf.contrib.rnn.GRUCell(num_units//2)
        cell_bw = tf.contrib.rnn.GRUCell(num_units//2)
        outputs, _ = tf.nn.bidirectional_dynamic_rnn(cell,cell_bw,inputs,sequence_length=seqlen,dtype=tf.float32)

        return tf.concat(outputs,2)

In [11]:
def highway(inputs,num_units,scope='highway'):
    with tf.variable_scope(scope):
        H = tf.layers.dense(inputs,units=num_units,activation=tf.nn.relu,name='H')
        T = tf.layers.dense(inputs,units=num_units,
                            activation=tf.nn.sigmoid,name='T',bias_initializer=tf.constant_initializer(-1.0))
        
        return H*T + inputs*(1.0)  

In [12]:
class cbhg():
    #inputs,is_training,scope
    def __init__(self,arg):
        tf.reset_default_graph()
        self.is_training = arg.is_training
        self.lr = arg.lr
        self.num_highway = arg.num_highway
        
        self.x = tf.placeholder(tf.float32,shape=(None,665, 256))
        self.y = tf.placeholder(tf.float32,shape=(None,665, 80))
        print(self.x.shape)
        #inputs shape:[Batch,T,256]
        
        inputs = tf.layers.dense(self.x,units=128,activation=None,use_bias=False,name='trans')

        convbanks_outputs = conv1d_banks(inputs,K=16,num_units=128,is_training=self.is_training)
        #output shape:[Batch,T,128*16]

        maxpool_outputs = tf.layers.max_pooling1d(convbanks_outputs,pool_size = 2,strides = 1,padding='same')
        #output shape:[Batch,T,128*16]

        c1 = Conv1d(inputs = maxpool_outputs,filters = 128,size = 3,activation=tf.nn.relu, scope='conv1d_1')
        #output shape:[Batch,T,128]

    #         x = normalize(x,is_training=self.is_training,activation_fn=None, scope="norm1")
        n1 = tf.layers.batch_normalization(c1,training = self.is_training)
        #output shape:[Batch,T,128]

        c2 = Conv1d(inputs = n1, filters = 128, size = 3, activation=None, scope='conv1d_2')
        #output shape:[Batch,T,128]

    #         x = normalize(x, is_training=self.is_training, activation_fn=None, scope="norm2")
        n2 = tf.layers.batch_normalization(c2,training = self.is_training)

        #output shape:[Batch,T,128]

        highway_inputs  = n2 + inputs
        #output shape:[Batch,T,128]

        for i in range(self.num_highway):
            highway_inputs = highway(highway_inputs,num_units=128,scope='highwaynet_{}'.format(i))
        #output shape:[Batch,T,128]

        bigru_outputs = gru_bi(inputs = highway_inputs,num_units = 128,scope='gru1')
        #output shape:[Batch,T,128]

        outputs = tf.layers.dense(bigru_outputs,units=80,activation=None,use_bias=False,name='O')
        #output shape:[Batch,T,80]
        
        if self.is_training:
            self.loss = tf.square(tf.subtract(outputs, self.y, name=None), name=None)
#             print(self.loss.shape)
            
            self.istarget = tf.to_float(tf.not_equal(self.y, tf.zeros_like(self.y))) # masking
#             print(self.istarget)
            
            self.mean_loss = tf.reduce_sum(self.loss * self.istarget) / 64.0
#             print(self.mean_loss.shape)
            
            self.global_step = tf.Variable(0,name='global_step',trainable=False)
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
            self.train_op = self.optimizer.minimize(self.mean_loss, global_step=self.global_step)
            
            tf.summary.scalar('loss',self.mean_loss)
            self.merged = tf.summary.merge_all()

In [13]:
def create_hparams():
    params = tf.contrib.training.HParams(
        lr = 0.001,
        num_highway = 4,
        is_training = True)
    
    return params

arg = create_hparams()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import time

start = time.time()
epochs = 10000
batch_size = 64

g = cbhg(arg)

saver = tf.train.Saver()
import warnings
warnings.filterwarnings('ignore')

with tf.Session() as sess:
    merged = tf.summary.merge_all()
    sess.run(tf.global_variables_initializer())
    if os.path.exists('logs/model.meta'):
        saver.restore(sess, './logs/model')
        
    writer = tf.summary.FileWriter('tensorboard/lm', tf.get_default_graph())
    
    for k in range(epochs):
        total_loss =  0
        #input_batch
        batch_num = len(input_batch) // batch_size
        batch = get_batch(input_batch, target_batch, batch_size)
        for i in range(batch_num):
            input_batch,target_batch = next(batch)
            print(input_batch.shape)
            print(target_batch.shape)
            feed = {g.x:input_batch,g.y:target_batch}
            cost,_ = sess.run([g.mean_loss,g.train_op],feed_dict=feed)
            total_loss += cost
            if (k * batch_num + i) % 10 == 0:
                rs = sess.run(merged, feed_dict=feed)
                writer.add_summary(rs, k * batch_num + i)
                
        print('epochs', k+1, ':average loss = ', total_loss/batch_num)
    
    saver.save(sess, './logs/model')
    writer.close()
end = time.time()
print(f'训练时间:{(end-start)/60:.2f}')

(?, 665, 256)
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.


Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.MaxPooling1D instead.


Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.cast` instead.
(64, 665, 256)
(64, 665, 80)
epochs 1 :average loss =  92994.6640625
(64, 665, 256)
(64, 665, 80)
